# Simple Transfer Learning Example
The example demostrates how to build a new model on top of an existing model taking advantage of an existing model via transfler learning

## Transfer Learning on an existing own model

In [1]:
# Load the target model
from keras.models import load_model
base_model = load_model('../models/simple-cifar10.h5')
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model
from keras.layers import Input
x = base_model.output
x = Flatten(name='flatten_1')(x)
x = Dense(1024, activation='relu', name='dense_x')(x)
x = Dropout(0.5, name='dropout_x')(x)
predictions = Dense(10, activation='softmax', name='predictions')(x)
model = Model(inputs=base_model.input, outputs=predictions)

2024-06-29 16:43:36.407137: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-29 16:43:36.407201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-29 16:43:36.428346: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


we can now use the model above to train further depending on our needs

## Finetune a popular 3rd party model (VGG16)
A more typical example will be to fine tune a popular 3rd party model to benefit from other people's work. We will show how to do this with the popular VGG16 model and the CIFAR-10 dataset. The model is not of high accuracy and is to demonstarate the techniques evolved. You may need to add more layers or/and upscale the cifar-10 images to get good results  

In [10]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Flatten, Dense, Dropout,GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array, array_to_img
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np

Load an prepate our CIFAR-10 data

In [6]:
# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Convert class vectors to binary class matrices
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

Download the pre-trained VGG16 model and create a new model on top of it

In [7]:
# Load the VGG16 model, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Use global average pooling to reduce the feature maps
x = Dense(1024, activation='relu', name='dense_x')(x)
x = Dropout(0.5, name='dropout_x')(x)
predictions = Dense(10, activation='softmax', name='predictions')(x)

# Create the new model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
# Train and evaluate the model
batch_size = 32
epochs = 10

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), verbose=2)

# Evaluate the model
scores = model.evaluate(x_test, y_test, verbose=1)
print(f"Test loss: {scores[0]}")
print(f"Test accuracy: {scores[1]}")

Epoch 1/10
1563/1563 - 6s - loss: 0.8420 - accuracy: 0.7062 - val_loss: 1.0444 - val_accuracy: 0.6385 - 6s/epoch - 4ms/step
Epoch 2/10
1563/1563 - 6s - loss: 0.8369 - accuracy: 0.7082 - val_loss: 1.0459 - val_accuracy: 0.6392 - 6s/epoch - 4ms/step
Epoch 3/10
1563/1563 - 6s - loss: 0.8260 - accuracy: 0.7133 - val_loss: 1.0421 - val_accuracy: 0.6407 - 6s/epoch - 4ms/step
Epoch 4/10
1563/1563 - 6s - loss: 0.8227 - accuracy: 0.7146 - val_loss: 1.0448 - val_accuracy: 0.6411 - 6s/epoch - 4ms/step
Epoch 5/10
1563/1563 - 6s - loss: 0.8156 - accuracy: 0.7184 - val_loss: 1.0400 - val_accuracy: 0.6407 - 6s/epoch - 4ms/step
Epoch 6/10
1563/1563 - 6s - loss: 0.8100 - accuracy: 0.7204 - val_loss: 1.0429 - val_accuracy: 0.6400 - 6s/epoch - 4ms/step
Epoch 7/10
1563/1563 - 6s - loss: 0.8040 - accuracy: 0.7223 - val_loss: 1.0366 - val_accuracy: 0.6434 - 6s/epoch - 4ms/step
Epoch 8/10
1563/1563 - 6s - loss: 0.7967 - accuracy: 0.7230 - val_loss: 1.0435 - val_accuracy: 0.6431 - 6s/epoch - 4ms/step
Epoch 9/